# 한국어 BERT 모델 학습

* task 해결 과정
  * dataset 생성
  * dataloader 생성
  * trainer argument 채워줌
  * train 함

## 학습 데이터 가져오기

In [ ]:
!mkdir my_data

* 개인적으로 데이터 준비

* 학습 준비

In [ ]:
!pip install transformers

### Tokenizer 생성

In [ ]:
!pip install transformers

* `BertWordPieceTokenizer` class를 만들고 train을 하게 되면 사용할 수 있는 Word Piece Toeknizer를 바로 획득 가능

In [ ]:
from tokenizers import BertWordPieceTokenizer

# Initialize an empty tokenizer
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True,   # ["이순신", "##은", " ", "조선"] ->  ["이순신", "##은", "조선"]
    # if char == " " or char == "\t" or char == "\n" or char == "\r":
    handle_chinese_chars=True,  # 한자는 모두 char 단위로 쪼게버립니다.
    strip_accents=False,    # True: [YehHamza] -> [Yep, Hamza]
    lowercase=False,    # Hello -> hello
)

# And then train
wp_tokenizer.train(
    files="my_data/wiki_20190620_small.txt",
    vocab_size=20000,   # vocab size 를 지정해줄 수 있습니다.
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##"
)

# Save the files
wp_tokenizer.save_model("wordPieceTokenizer", "my_tokenizer") # ['wordPieceTokenizer/my_tokenizer-vocab.txt']

In [ ]:
print(wp_tokenizer.get_vocab_size()) # 20000

In [ ]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = wp_tokenizer.encode(text)
print(tokenized_text.tokens)
# ['이', '##순', '##신은', '조선', '중', '##기의', '무신', '##이다', '.']
print(tokenized_text.ids)
# [704, 1346, 7588, 2001, 753, 2603, 13158, 1896, 16]

## BERT 학습

* BERT 껍데기을 만든 후 dataset을 dataloader를 통해서 계속 먹여줌으로써 껍데기의 weight를 조절해주며 학습함

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast(
    vocab_file='/content/wordPieceTokenizer/my_tokenizer-vocab.txt',
    max_len=128,
    do_lower_case=False,
    )

* 데이터셋을 만들 때 [MASK] token을 부착하여 넘길수도 있음
  * special token으로 등록되지 않은 경우 쪼개질 수 있음

In [ ]:
print(tokenizer.tokenize("뷁은 [MASK] 조선 중기의 무신이다."))
# ['[UNK]', '[', 'M', '##AS', '##K', ']', '조선', '중', '##기의', '무신', '##이다', '.']

* [MASK] token 을 special token에 추가

In [ ]:
tokenizer.add_special_tokens({'mask_token':'[MASK]'})
print(tokenizer.tokenize("이순신은 [MASK] 중기의 무신이다."))
# ['이', '##순', '##신은', '[MASK]', '중', '##기의', '무신', '##이다', '.']

* BERT 껍데기 생성
  * configuration을 통해 조건을 조절함
    * `vocab_size`
      * default는 영어기준으로 되어있기 때문에 반드시 수정이 필요함
    * `hidden_size` : tranformer의 hidden vector size
      * 동작의 빠른 속도가 필요한 경우 값을 줄여서 효과를 기대해볼 수 있음
    * `num_hidden_layers` : 쌓고자 하는 hidden layer 개수
      * BERT 같은 pretrained model을 사용하지만, 좀 더 빠른 상태에서 동작하길 원하는 경우 layer 수를 줄여도(3, 6 정도) 성능 차이가 적음
    * `num_attention_heads` : transformer의 multi head self-attention의 개수
    * `intermediate_size` : transformer 내에 있는 feed-forward network의 dimension size
    * `hidden_act` : activation function
    * `hidden_dropout_prob` : dropout 정보
    * `max_position_embeddings` : model이 받을 수 있는 input token의 최대 size
      * default는 512
      * 댓글 분석인경우 32, 64 정도에서 대부분 처리 가능함
      * task가 장문으로 되어있는 경우 1024 로 설정할 수도 있음
    * `type_vocab_size` : type id 범위
      * BERT는 segmentA, segmentB로 구분되어 있기 때문에 2(종류)로 정의
    * `pad_token_id` : tokenizer에서 pad token이 가지는 id
    * `position_embedding_type`
      * 'absolute' : input token의 위치에 따라 절대값으로 positional embedding을 함

* `BertForPreTraining`
  * transformers에서 제공하는 model type
  * 정의한 configuration을 넣어주면 model 껍데기가 생성됨

In [ ]:
from transformers import BertConfig, BertForPreTraining

config = BertConfig(    # https://huggingface.co/transformers/model_doc/bert.html#bertconfig
    vocab_size=20000,
    # hidden_size=512,
    # num_hidden_layers=12,    # layer num
    # num_attention_heads=8,    # transformer attention head number
    # intermediate_size=3072,   # transformer 내에 있는 feed-forward network의 dimension size
    # hidden_act="gelu",
    # hidden_dropout_prob=0.1,
    # attention_probs_dropout_prob=0.1,
    max_position_embeddings=128,    # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정
    # type_vocab_size=2,    # token type ids의 범위 (BERT는 segmentA, segmentB로 2종류)
    # pad_token_id=0,
    # position_embedding_type="absolute"
)

model = BertForPreTraining(config=config)
model.num_parameters() # 101720098

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from typing import Dict, List, Optional
import os
import json
import os
import pickle
import random
import time
import warnings

from filelock import FileLock

from transformers.utils import logging

logger = logging.get_logger(__name__)


* dataset 구성
  * document 단위로 학습이 이루어짐
    * ex. 2개의 document가 서로 다른 인물에 대한 정보일 경우 첫 번째 문서의 마지막 문장과 두 번째 문장의 첫번째 문장이 next sentence prediction으로 연결되지 않도록 방지

In [ ]:
class TextDatasetForNextSentencePrediction(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.1,
        nsp_probability=0.5,
    ):
        # 여기 부분은 학습 데이터를 caching하는 부분입니다 :-)
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=True) # max embedding token size
        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            "cached_nsp_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        self.tokenizer = tokenizer

        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else: # cash가 없는 경우
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 dataset을 만듭니다.
                self.documents = [[]] # document 단위로 학습이 이루어짐
                with open(file_path, encoding="utf-8") as f:
                    while True: # 일단 문장을 읽고
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip() # 공백 및 문장 바꾸는 기호 제거

                        # 이중 띄어쓰기가 발견된다면, 나왔던 문장들을 모아 하나의 문서로 묶어버립니다.
                        ## documents 마지막에 token을 append함
                        # 즉, 문단 단위로 데이터를 저장합니다.
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([]) 
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)
                # 이제 코퍼스 전체를 읽고, 문서 데이터를 생성했습니다! :-)
                logger.info(f"Creating examples from {len(self.documents)} documents.") # examples : 데이터에 사용되는 데이터 덩어리
                self.examples = []
                # 본격적으로 학습을 위한 데이터로 변형시켜볼까요?
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index) # 함수로 가봅시다. # 최종 dataset이 만들어짐

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def create_examples_from_document(self, document: List[List[int]], doc_index: int):
        """Creates examples for a single document."""
        # 문장의 앞, 뒤에 [CLS], [SEP] token이 부착되기 때문에, 내가 지정한 size에서 2 만큼 빼줍니다.
        # 예를 들어 128 token 만큼만 학습 가능한 model을 선언했다면, 학습 데이터로부터는 최대 126 token만 가져오게 됩니다.
        max_num_tokens = self.block_size - self.tokenizer.num_special_tokens_to_add(pair=True)

        # We *usually* want to fill up the entire sequence since we are padding
        # to `block_size` anyways, so short sequences are generally wasted
        # computation. However, we *sometimes*
        # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
        # sequences to minimize the mismatch between pretraining and fine-tuning.
        # The `target_seq_length` is just a rough target however, whereas
        # `block_size` is a hard limit.

        # 여기가 재밌는 부분인데요!
        # 위에서 설명했듯이, 학습 데이터는 126 token(128-2)을 채워서 만들어지는게 목적입니다.
        # 하지만 나중에 BERT를 사용할 때, 126 token 이내의 짧은 문장을 테스트하는 경우도 분명 많을 것입니다 :-)
        ## 512 token을 다 채워서 학습한 BERT model의 경우 짧은 문장을 처리할 능력이 없을 수 있음
        # 그래서 short_seq_probability 만큼의 데이터에서는 2-126 사이의 random 값으로 학습 데이터를 만들게 됩니다.
        ## 더 다양한 데이터를 반영해서 처리 가능함
        target_seq_length = max_num_tokens # BERT가 학습할 때, segmentA와 segmentB가 합쳐져서 학습됨
        if random.random() < self.short_seq_probability: # random.random() : 0.0 ~ 1 사이의 값으로 return
            target_seq_length = random.randint(2, max_num_tokens)

        current_chunk = []  # a buffer stored current working segments
        current_length = 0
        i = 0

        # 데이터 구축의 단위는 document 입니다
        # 이 때, 무조건 문장_1[SEP]문장_2 이렇게 만들어지는 것이 아니라,
        # 126 token을 꽉 채울 수 있게 문장_1+문장_2[SEP]문장_3+문장_4 형태로 만들어질 수 있습니다.
        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` is how many segments from `current_chunk` go into the `A`
                    # (first) sentence.
                    a_end = 1
                    # 여기서 문장_1+문장_2 가 이루어졌을 때, 길이를 random하게 짤라버립니다 :-)
                    ## 문장을 자른 후 [SEP] 부착하고 다음 문장을 나열함
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)
                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])
                    # 이제 [SEP] 뒷 부분인 segmentB를 살펴볼까요?
                    tokens_b = []
                    # 50%의 확률로 랜덤하게 다른 문장을 선택하거나, 다음 문장을 학습데이터로 만듭니다.
                    if len(current_chunk) == 1 or random.random() < self.nsp_probability:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be careful, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break
                        # 여기서 랜덤하게 선택합니다 :-)
                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        # We didn't actually use these segments so we "put them back" so
                        # they don't go to waste.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Actual next
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    # 이제 126 token을 넘는다면 truncation을 해야합니다.
                    # 이 때, 126 token 이내로 들어온다면 행위를 멈추고,
                    # 만약 126 token을 넘는다면, segmentA와 segmentB에서 랜덤하게 하나씩 제거합니다.
                    ## truncation rule을 단순히 뒷부분 자르는 것으로 지정한다면, segmentA와 segmentB의 비율 차이가 심하거나
                    ## [SEP]이내로 잘릴수도 있음
                    def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
                        """Truncates a pair of sequences to a maximum sequence length."""
                        while True:
                            total_length = len(tokens_a) + len(tokens_b)
                            if total_length <= max_num_tokens:
                                break
                            trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                            assert len(trunc_tokens) >= 1
                            # We want to sometimes truncate from the front and sometimes from the
                            # back to add more randomness and avoid biases.
                            if random.random() < 0.5:
                                del trunc_tokens[0]
                            else:
                                trunc_tokens.pop()

                    truncate_seq_pair(tokens_a, tokens_b, max_num_tokens)

                    assert len(tokens_a) >= 1
                    assert len(tokens_b) >= 1

                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b) # vocab id
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)
                    
                    # 드디어 아래 항목에 대한 데이터셋이 만들어졌습니다! :-)
                    # 즉, segmentA[SEP]segmentB, [0, 0, .., 0, 1, 1, ..., 1], NSP 데이터가 만들어진 것입니다 :-)
                    # 그럼 다음은.. 이 데이터에 [MASK] 를 씌워야겠죠?
                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

* `DataCollatorForLanguageModeling` : 정의한 mask propability에 맞추어 masking함

In [ ]:
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path='[/content/my_data/wiki_small.txt]',
    block_size=128,
    overwrite_cache=False,
    short_seq_probability=0.1,
    nsp_probability=0.5,
)

data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

* 데이터 확인

In [ ]:
for example in dataset.examples[0:1]:
    print(example)

## 2 : [CLS], 3 : [SEP]
'''
{'input_ids': tensor([    2,     5,  5504,  9439,  2489,  2428,  2779,  1968,  5379,  3111,
         1940,  2407,    16,  5497, 10310, 16250,   553,  1073,   822,  1464,
         1217,   931, 16494, 12290,  1042,  3666,    16,  6528,  8936,  1022,
         2677,  1906,    16,   174,   985,  4021,  1019,  8598,   728,  1271,
           93,  7743,    93, 10414,  1063, 18368,  3503, 18888,    16,  6435,
         1968,  4021,   277,  3361,   658,  1195,  2105,  1933, 17664,    93,
          437,  1155,     3,   494,  2736,     5, 17664, 12976,     5,   377,
         7719,    16,  4186,  6528,   750,   542,  1256,  2795,  4859,  5152,
         4769,   174, 11846, 15561,   655,  2786,  9395,  1945,  2370,  2895,
         2053,    14,  9874,  6528,   750,   762,  1061,  6459,  5152,  2823,
         3950,  6528,   750,   762,  2092,  6204,  1899,    16,  2829,  4530,
          728, 16250, 18217,  2665, 17627, 13573,  2492,    14,  3951,  1916,
        16017,  6528,   762,  2873,     3]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1]), 'next_sentence_label': tensor(0)}
'''

[MASK]를 부착하는 data collator의 역할 확인
  * labels : mask된 token의 원래 id를 알려주는 역할을 함

In [ ]:
print(data_collator(dataset.examples))
'''
{'input_ids': tensor([[    2,     4,  5504,  ...,   762,  2873,     3],
        [    2,  6528,  7895,  ...,     0,     0,     0],
        [    2,  6793,  1900,  ...,  1895,    16,     3],
        ...,
        [    2,  3953,  3262,  ...,     4,    16,     3],
        [    2,     4,  2412,  ...,  1913,     4,     3],
        [    2, 12577, 10891,  ...,  1047,  2833,     3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'next_sentence_label': tensor([0, 1, 1,  ..., 1, 1, 1]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[-100,    5, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., 1887, -100, -100],
        [-100, 1988, -100,  ..., -100,   16, -100], # 특정 token이 16번째 vocab token으로 replace된 것을 알려줌
        [-100, -100, -100,  ..., -100, -100, -100]])}
'''

In [ ]:
print(data_collator(dataset.examples)['input_ids'][0])
'''
tensor([    2,     5,  5504,  9439,     4,  2428,  2779,  1968,  5379,  3111,
         1940,     4,    16,  5497, 10310, 16250,   553,  1073,   822,  1464,
         1217,   931, 16494, 12290,  1042,  3666,    16,  6528,  8936,  1022,
         6038,     4,    16,   174,   985,  4021,  1019,  8598,   728,  1271,
           93,  7743,    93, 10414,     4, 18368,  3503, 18888,    16,  6435,
         1968,  4021,   277,  3361,   658,  1195,  2105,  1933, 17664,    93,
          437,  1155,     3,   494,  2736,     4, 17664, 12976,     4,   377,
         7719,    16,  4186,  6528,   750,   542,  1256,  2795,  4859,  5152,
         4769,   174, 11846,     4,   655,  2786,  9395,     4,  2370,  2895,
            4,    14,  9874,  6528,   750,   762,  1061,  6459,  5152,  2823,
         3950,  6528,   750,   762,  2092,     4,  1899,    16,  2829,  4530,
          728, 16250, 18217,  2665, 17627, 13573,  2492,    14,  3951,  1916,
        16017,  6528,   762,     4,     3])
'''

* decoding하여 관찰

In [ ]:
tokenizer.decode(data_collator(dataset.examples)['input_ids'][0].tolist())
'''
'[CLS] 주니어는 민주당 출신 미국 [MASK]번째 대통령 이다. 지미 카터는 [MASK] 섬 [MASK] 카운 [MASK] 플레인스 마을에서 [MASK]. 조지아 공과대학교를 졸업 [MASK]구와 그 후 해군에대백과사전 전함 · 원자력 · 잠수함의 승무원으로 일하였다. 1953년 미국 해군 대 [MASK] 예편하였고 이후 땅콩 경선과 면화 등을 [UNK] 많은 돈을 벌었다 [MASK] 그의 별명이 " [SEP] 와 같이 쓸 수 있다. 그런데 formula _ 27은 formula _ 28차의 [MASK]므로, 다항식의 차수에 대한 귀납법을위원회 증명이 끝난다. [MASK] formula _ 19차 다항식의 경우, 위의 따름정리를 적용하면 이 역시 복소수체 위에서 중근을 [MASK] 경우 formula _ 19개의 근을 갖는다. [SEP]
'''

* `Trainer`
  * transformers에서 제공하는 train 기능
    * `output_dir` : model이 output되는 위치
    * `overwrite_output_dir` : model이 학습되면서 overwrite가 될건지에 대한 여부
    * `num_train_epochs` : 주어진 데이터를 대상으로 주는 epoch 수
    * `per_gpu_train_batch_size`
    * `save_steps` : 저장 기준이 되는 step 수
      * 1000 : 1000 step마다 데이터를 저장함
    * `save_total_limit` : 데이터를 저장하는 최대 개수
      * 2 : 마지막 2개의 데이터를 제외하고 이전 데이터들을 제거함
    * `logging_steps` : log를 찍어주는 기준이 되는 step 수
      * 100 : 100 step마다 log를 찍어줌

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train() # wiki 전체 데이터로 학습 시, 1 epoch에 9시간 정도 소요됩니다!! 

* 특정 directory로 model을 저장함

In [ ]:
trainer.save_model('./model_output')

* [MASK] 테스트
  * 실습에서 사용된 model은 작고 빠르게 학습했기 때문에 성능이 좋지는 않음

In [ ]:
from transformers import BertForMaskedLM, pipeline

* [MASK] task를 위해서 model을 다르게 load함

* `BertForMaskedLM`을 사용하여 학습한 model 가져오기

In [ ]:
my_model = BertForMaskedLM.from_pretrained('model_output')

* tokenizer 확인

In [ ]:
tokenizer.tokenize('이순신은 [MASK] 중기의 무신이다.')
# ['이', '##순', '##신은', '[MASK]', '중', '##기의', '무신', '##이다', '.']

* `pipeline` 생성
  * model, tokenizer 명시
  * GPU를 사용하는 경우 device 명시해야함
    * 0번 GPU를 사용하는 경우 `device=0`으로 명시
    * 명시하지 않으면 CPU로 동작함

In [ ]:
nlp_fill = pipeline('fill-mask', top_k=5, model=my_model, tokenizer=tokenizer)

In [ ]:
nlp_fill('이순신은 [MASK] 중기의 무신이다.')
'''
[{'score': 0.030770400539040565,
  'sequence': '[CLS] 이순신은, 중기의 무신이다. [SEP]',
  'token': 14,
  'token_str': ','},
 {'score': 0.03006444126367569,
  'sequence': '[CLS] 이순신은. 중기의 무신이다. [SEP]',
  'token': 16,
  'token_str': '.'},
 {'score': 0.012540608644485474,
  'sequence': '[CLS] 이순신은 _ 중기의 무신이다. [SEP]',
  'token': 63,
  'token_str': '_'},
 {'score': 0.008801406249403954,
  'sequence': '[CLS] 이순신은 있다 중기의 무신이다. [SEP]',
  'token': 1888,
  'token_str': '있다'},
 {'score': 0.008582047186791897,
  'sequence': '[CLS] 이순신은 formula 중기의 무신이다. [SEP]',
  'token': 1895,
  'token_str': 'formula'}]
'''

In [ ]:
nlp_fill('[MASK]는 조선 중기의 무신이다.')
'''

[{'score': 0.025915304198861122,
  'sequence': '[CLS]. 는 조선 중기의 무신이다. [SEP]',
  'token': 16,
  'token_str': '.'},
 {'score': 0.024867292493581772,
  'sequence': '[CLS], 는 조선 중기의 무신이다. [SEP]',
  'token': 14,
  'token_str': ','},
 {'score': 0.008652042597532272,
  'sequence': '[CLS]에 는 조선 중기의 무신이다. [SEP]',
  'token': 1018,
  'token_str': '##에'},
 {'score': 0.008601967245340347,
  'sequence': '[CLS] formula 는 조선 중기의 무신이다. [SEP]',
  'token': 1895,
  'token_str': 'formula'},
 {'score': 0.008583025075495243,
  'sequence': '[CLS] 이 는 조선 중기의 무신이다. [SEP]',
  'token': 704,
  'token_str': '이'}]
'''